In [115]:
import pymongo as pm
import requests as rq
import ujson as js
import mysql.connector as mysql
from multiprocessing.dummy import Pool

In [ ]:
class LocationParser:
    def __init__(self, char_id, maria_cur, mongo_cur, url_cur):
        self.char_id = char_id
        self.cur = {
            'maria': maria_conn,
            'mongo': mongo_conn,
            'url': url_conn
        }
        
    mongo = {
        'login_path': '../settings/mongo_login.json',
        'database': 'NewEdenAnalytics',
        'collections': {
            'characters': 'eve_characters'
        }
    }
    maria = {
        'login_path': '../settings/maria_login.json',
        'queries': {
            'select': {
                'type': """\
                SELECT type_name,
                    mass
                FROM Types
                WHERE Types.type_id = {type_id}
                ;"""
            }
        }
    }
    scopes = {
        'online': 'esi-location.read_online.v1',
        'location': 'esi-location.read_location.v1',
        'ship': 'esi-location.read_ship_type.v1'
    }
    url = {
        'paths': {
            'root': 'https://esi.evetech.net/',
            'online': 'latest/characters/{char_id}/online/',
            'location': 'latest/characters/{char_id}/location/',
            'ship': 'latest/characters/{char_id}/ship/'
        },
        'headers': {'accept': 'application/json'},
        'params': {'datasource': 'tranquility'}
    }

In [110]:
class WormholeMapper:
    mongo = {
        'login_path': '../settings/mongo_login.json',
        'database': 'NewEdenAnalytics',
        'collections': {
            'characters': 'eve_characters'
        }
    }
    maria = {
        'login_path': '../settings/maria_login.json',
        'queries': {
            'select': {
                'type': """\
                    SELECT type_name,
                        mass
                    FROM Types
                    WHERE Types.type_id = {type_id}
                ;"""
            }
        }
    }
    scopes = {
        'online': 'esi-location.read_online.v1',
        'location': 'esi-location.read_location.v1',
        'ship': 'esi-location.read_ship_type.v1'
    }
    url = {
        'paths': {
            'root': 'https://esi.evetech.net/',
            'online': 'latest/characters/{char_id}/online/',
            'location': 'latest/characters/{char_id}/location/',
            'ship': 'latest/characters/{char_id}/ship/'
        },
        'headers': {'accept': 'application/json'},
        'params': {'datasource': 'tranquility'}
    }
    
    def __init__(self):
        self.conn = {}
        self.chars = {}
        
        self.connect_mongo()
        self.connect_maria()
        self.connect_url()
        
    def connect_mongo(self):
        with open(self.mongo['login_path']) as file:
            self.conn['mongo'] = pm.MongoClient(**js.load(file))
            
    def connect_maria(self):
        with open(self.maria['login_path']) as file:
            self.conn['maria'] = mysql.connect(**js.load(file))
            
    def connect_url(self):
        self.conn['url'] = rq.Session()
    
    def update_char_data(self):
        cursor = self.conn['mongo'][
            self.mongo['database']
        ][
            self.mongo['collections']['characters']
        ]
        for char in cursor.find({'tokens.scopes': {'$all': list(self.scopes.values())}}):
            if char['_id'] not in self.chars.keys():
                self.chars[char['_id']] = {
                    'char_name': char['char_name'],
                    'corp_id': char['corp_id'],
                    'access_token': char['tokens']['access_token']
                }
            
            self.update_online_status(char['_id'])
            
            if self.chars[char['_id']]['online']:
                self.update_location(char['_id'])
                
    def update_online_status(self, char_id):
        online_status = js.loads(self.get_url_data(
            'online',
            {'char_id': str(char_id)},
            headers = {'Authorization': 'Bearer %s' % self.chars[char_id]['access_token']}
        ).content)

        self.chars[char_id] = {
            **self.chars[char_id],
            **online_status
        }
    
    def update_location(self, char_id):
        location = js.loads(self.get_url_data(
            'location',
            {'char_id': str(char_id)},
            headers = {'Authorization': 'Bearer %s' % self.chars[char_id]['access_token']}
        ).content)
        
        ship = js.loads(self.get_url_data(
            'ship',
            {'char_id': str(char_id)},
            headers = {'Authorization': 'Bearer %s' % self.chars[char_id]['access_token']}
        ).content)
        
        ship_data = self.get_maria_data(mapper.maria['queries']['select']['type'], {'type_id': ship['ship_type_id']})[0]
        ship = {
            'type_id': ship['ship_type_id'],
            'name': ship['ship_name'],
            'item_id': ship['ship_item_id'],
            'type_name': ship_data[0][0],
            'mass': ship_data[1]
        }
        
        self.chars[char_id] = {
            **self.chars[char_id],
            'location': location['solar_system_id'],
            'ship': ship
        }
                
    def get_url_data(self, url_type, url_params, headers = {}, params = {}):
        url_return = self.conn['url'].get(
            '%s%s' % (
                self.url['paths']['root'],
                self.url['paths'][url_type].format(**url_params)
            ),
            headers = {
                **self.url['headers'],
                **headers
            },
            params = {
                **self.url['params'],
                **params
            }
        )
        
        return url_return
    
    def get_maria_data(self, query, query_params):
        cur = mapper.conn['maria'].cursor()
        cur.execute(query.format(**query_params))
        data = [row for row in cur]
        cur.close()
        return data
    
    def close_conns(self):
        for value in self.conn.values():
            value.close()

In [111]:
mapper = WormholeMapper()

In [112]:
mapper.update_char_data()

In [113]:
mapper.chars[144981811]

{'char_name': 'Calvin',
 'corp_id': 98479140,
 'access_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6IkpXVC1TaWduYXR1cmUtS2V5IiwidHlwIjoiSldUIn0.eyJzY3AiOlsiZXNpLWxvY2F0aW9uLnJlYWRfbG9jYXRpb24udjEiLCJlc2ktbG9jYXRpb24ucmVhZF9zaGlwX3R5cGUudjEiLCJlc2ktZmxlZXRzLnJlYWRfZmxlZXQudjEiLCJlc2ktZmxlZXRzLndyaXRlX2ZsZWV0LnYxIiwiZXNpLXVpLndyaXRlX3dheXBvaW50LnYxIiwiZXNpLWxvY2F0aW9uLnJlYWRfb25saW5lLnYxIiwiZXNpLWluZHVzdHJ5LnJlYWRfY2hhcmFjdGVyX21pbmluZy52MSJdLCJqdGkiOiJjZTg2ODdkNC1kNDBiLTRhMTQtODA4Yy0zM2YzNmZjZmVhMjMiLCJraWQiOiJKV1QtU2lnbmF0dXJlLUtleSIsInN1YiI6IkNIQVJBQ1RFUjpFVkU6MTQ0OTgxODExIiwiYXpwIjoiYjlmYWM1YmI3YzViNGZlODk3MjVlN2UzMWM0MDEyNjkiLCJuYW1lIjoiQ2FsdmluIiwib3duZXIiOiJpQU1Hc3A1TUVzTE5HNXo1aGRwcHhMQ0JoY1E9IiwiZXhwIjoxNTY1NTQyMjAyLCJpc3MiOiJsb2dpbi5ldmVvbmxpbmUuY29tIn0.nJ1JQEWSb7CEuWAtWpqXIte23-klq3-xVfuj4k5I8uIu5euE1i1lfdEa4afq5A_hwQVdjnPFwk-eLcEdA--7pO-suSxP3HlB1XtTX15ebAzkwc-uFyOaQuKSWyhfIWkp1XTOMudbrZjhrF57X7d4c0S2BrlwotsbZmhz4D2ukbJKU24wF0ndfajeSfhWixLrFdPLS_NKJL8OaAUPdpKn4_lQ_EDhpm-pQ8d8UMTnkbdu